In [1]:
from src.FunctionsSurface import *
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm  # Import LogNorm

In [ ]:
N = 5000
dim = [1,2,3,4]

parms_1d,parms_2d,parms_3d,parms_4d = [],[],[],[]

for i in dim:
    parms = [k for k in list_all_folders(N,i)]
    for l in range(len(parms)):
        tuple = (float(parms[l][0]),float(parms[l][1]))
        if(i == 1):
            parms_1d.append(tuple)
        elif(i == 2):
            parms_2d.append(tuple)
        elif(i==3):
            parms_3d.append(tuple)
        elif(i==4):
            parms_4d.append(tuple)

In [ ]:
alpha_a = np.arange(0.0, 9.1, 0.1)
alpha_g = np.arange(0.0, 9.1, 0.1)
theoric = []
for i in range(len(alpha_a)):
    for j in range(len(alpha_g)):
        theoric.append((round(alpha_a[i],2),round(alpha_g[j],2)))

In [ ]:
remening_1d = []
remening_2d = []
remening_3d = []
remening_4d = []

for i in range(len(theoric)):
    if theoric[i] not in parms_1d:
        remening_1d.append((theoric[i][0], theoric[i][1], 1))
    
    elif theoric[i] not in parms_2d:
        remening_2d.append((theoric[i][0], theoric[i][1], 2))
    
    elif theoric[i] not in parms_3d:
        remening_3d.append((theoric[i][0], theoric[i][1], 3))
    
    elif theoric[i] not in parms_4d:
        remening_4d.append((theoric[i][0], theoric[i][1], 4))

remening_all = remening_1d + remening_2d + remening_3d + remening_4d

dict_ = {"alpha_a":[],"alpha_g":[],"dim":[]}

for i in range(len(remening_all)):
    dict_["alpha_a"].append(remening_all[i][0])
    dict_["alpha_g"].append(remening_all[i][1])
    dict_["dim"].append(remening_all[i][2])
df = pd.DataFrame(data=dict_)

In [ ]:
df.to_csv("rest_parms.txt", index=False, sep=" ",mode="w+")

In [ ]:
df.head()

In [ ]:
N = 5000

def copy_files_cleber(N, dim, alpha_a, alpha_g):
    my_file = f"../../data/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/prop/properties_set.txt"
    new_path = f"../../../raw_N_{N}/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/prop"
    
    # Create folder with file
    if not os.path.exists(new_path):
        os.makedirs(new_path)
        # Copy file to /raw_N_{N}
        shutil.copy(my_file, new_path)
    else:
        pass


In [2]:
N = 5000
dim = [1,2,3,4]
for i in dim:
    parms =  [k for k in list_all_folders(N,i)]
    for l in range(len(parms)):
        all_properties_dataframe(N, i, parms[l][0], parms[l][1])

N=5000, dim = 1, alpha_a = 2.8, alpha_g = 5.7


OSError: Cannot save file into a non-existent directory: '../../data/N_5000/dim_1/alpha_a_2.8_alpha_g_5.7/prop'

In [ ]:
alpha_a = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
alpha_g = [0.1, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

for n in N__:
    for d in dim:
        for g in alpha_g:
            for a in alpha_a:
                alpha_G = list_all_folders_for_alpha_fixed(n, d , a, g, alpha_g_variable=True)
                create_all_properties_file(n, d, a, alpha_G, alpha_g_variable=True)

    for d in dim:
        for a in alpha_a:
            for g in alpha_g:
                alpha_A = list_all_folders_for_alpha_fixed(n, d , a, g, alpha_g_variable=False)
                create_all_properties_file(n, d, alpha_A, g, alpha_g_variable=False)

In [ ]:
data = pd.read_csv("../../data/all_data.csv",sep=',')
all_data =  filter_dataframe(data)
all_data.head()

In [ ]:
all_data["n_size"].unique()

In [ ]:
df__ = assortativity_N(all_data, [5000, 10000, 20000, 40000])

In [ ]:
median = []
avarage = []
for i in [1,2,3,4]:
    order = [find_order_of_magnitude(i) for i in df__[df__['dim']  == i ]["N*"].values]
    median.append(np.median(sorted(order)))
    avarage.append(np.mean(order))
    plt.hist(order,label=f'dim = {i}', alpha = 0.3, edgecolor='black')
print("median =",median)
print("mean = ",avarage)
plt.xlabel("Order of magnitude N*")
plt.legend()
plt.show()

In [ ]:
dim = 2
for dim in [1,2,3,4]:
        data = {'x': df__[df__["dim"] == dim]["alpha_a"].values,
                'y': df__[df__["dim"] == dim]["alpha_g"].values,
                'z': np.log10(df__[df__["dim"] == dim]["N*"].values)}

        df_ = pd.DataFrame(data)

        df_ = df_.sort_values(by='y')

        # Pivot the DataFrame to prepare for heatmap
        pivot_df = df_.pivot(index='y', columns='x', values='z')

        # Plotting the heatmap
        #ax = sns.heatmap(pivot_df, cmap='coolwarm', norm=LogNorm())
        ax = sns.heatmap(pivot_df, cmap='coolwarm', yticklabels=sorted(df_['y'].unique()))
        cbar = ax.collections[0].colorbar
        cbar.set_label(r"$\log_{10} N^*$", fontsize=18)

        # Adding title and labels
        plt.title(f'Dimension = {dim}')
        plt.xlabel(r'$\alpha_a$',fontsize=18)
        plt.ylabel(r'$\alpha_g$',fontsize=18)

        # Display the plot
        plt.show()

In [ ]:
all_data_filter = filter_dataframe(all_data)
all_data_filter.head()

In [ ]:
List_N = [5000, 10000, 20000, 40000]
df = beta_all(all_data_filter, List_N)
df.head()

In [ ]:
df_1d = df[df["dim"]==1]
df_2d = df[df["dim"]==2]
df_3d = df[df["dim"]==3]
df_4d = df[df["dim"]==4]

In [ ]:
def heatmap(dataframe, propertie = "beta_short", label_prop = r'$\beta_\ell$' ,dim = 1):
        # Example DataFrame with x, y, z values
        
        data = {'x': dataframe[dataframe["dim"] == dim]["alpha_a"].values,
                'y': dataframe[dataframe["dim"] == dim]["alpha_g"].values,
                'z': dataframe[dataframe["dim"] == dim][propertie].values}

        df_ = pd.DataFrame(data)
        
        df_ = df_.sort_values(by='y')
        
        # Pivot the DataFrame to prepare for heatmap
        pivot_df = df_.pivot(index='y', columns='x', values='z')
        
        # Plotting the heatmap
        #ax = sns.heatmap(pivot_df, cmap='coolwarm', norm=LogNorm())
        ax = sns.heatmap(pivot_df, cmap='coolwarm', yticklabels=sorted(df_['y'].unique()))
        cbar = ax.collections[0].colorbar
        cbar.set_label(label_prop, fontsize=18)
        
        # Adding title and labels
        plt.title(f'Dimension = {dim}')
        plt.xlabel(r'$\alpha_a$',fontsize=18)
        plt.ylabel(r'$\alpha_g$',fontsize=18)

        # Display the plot
        plt.show()

In [ ]:
for d in [1,2,3,4]:
    heatmap(df,  dim = d)